In [ ]:
import random
import numpy as np

# Tamaños y símbolos del Sudoku
base  = 3  # Base de la cuadrícula 3x3
side  = base * base
symbol = " 1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Expande las líneas de la cuadrícula visual
def expandLine(line):
    return line[0] + line[5:9].join([line[1:5]*(base-1)]*base) + line[9:13]

# Función para generar el tablero base
def pattern(r, c): return (base*(r % base) + r//base + c) % side

# Función para mezclar filas, columnas y números para aleatoriedad
def shuffle(s): return random.sample(s, len(s))

def create_board():
    # Generar la estructura base de Sudoku
    rBase = range(base)
    rows  = [g*base + r for g in shuffle(rBase) for r in shuffle(rBase)]
    cols  = [g*base + c for g in shuffle(rBase) for c in shuffle(rBase)]
    nums  = shuffle(range(1, base*base + 1))

    # Crear el tablero completo
    board = [[nums[pattern(r, c)] for c in cols] for r in rows]
    return board

# Función para eliminar números en base a la dificultad
def remove_numbers(board, difficulty):
    if difficulty == 'facil':
        empty_cells = 40
    elif difficulty == 'medio':
        empty_cells = 50
    elif difficulty == 'dificil':
        empty_cells = 60
    else:
        raise ValueError("Dificultad no válida. Elija entre 'facil', 'medio' o 'dificil'")

    # Eliminar números aleatoriamente
    for _ in range(empty_cells):
        row, col = random.randint(0, side-1), random.randint(0, side-1)
        board[row][col] = 0

    return board

# Función para imprimir el tablero en formato visual
def print_board(board):
    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    nums   = [[""] + [symbol[n] if n != 0 else " " for n in row] for row in board]
    print(line0)
    for r in range(1, side+1):
        print("".join(n+s for n, s in zip(nums[r-1], line1.split("."))))
        print([line2, line3, line4][(r % side == 0) + (r % base == 0)])

# Generar Sudoku
difficulty = 'dificil' #cambio de dificultad
board = create_board()
puzzle_board = remove_numbers(np.copy(board), difficulty)

# Imprimir el Sudoku generado
print("Sudoku generado:")
print_board(puzzle_board)


Sudoku generado:
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 3 │ 4 ║ 6 │   │   ║ 7 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │ 5 ║ 3 │   │ 8 ║ 6 │ 1 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 1 ║   │ 5 │ 9 ║   │ 4 │ 8 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 7 │ 4 ║ 2 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 2 │   │   ║ 9 │   │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 2 │ 3 ║   │   │   ║ 8 │ 7 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 4 │ 9 ║ 1 │ 8 │ 3 ║ 5 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 1 │ 8 ║   │   │ 6 ║ 4 │ 9 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │ 9 │   ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


In [ ]:
board =[[0, 2, 0, 0, 1, 3, 0, 6, 0],
 [0, 0, 5, 6, 0, 0, 3, 4, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 2, 0, 7, 0, 0, 8, 5],
 [0, 9, 0, 0, 0, 2, 0, 0, 0],
 [7, 0, 0, 0, 3, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 5, 9, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 5, 1],
 [0, 0, 0, 8, 0, 0, 0, 7, 0]]

#Backtracking

In [ ]:
import time
import numpy as np

# Función para verificar si un número puede colocarse en una posición del tablero
def is_valid(board, row, col, num):
    # Verificar fila
    if num in board[row]:
        return False
    # Verificar columna
    if num in [board[i][col] for i in range(9)]:
        return False
    # Verificar cuadrante
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

# Función de Backtracking para resolver el Sudoku
def solve_backtracking(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def backtrack(index):
        if index == len(empty_cells):
            return True
        row, col = empty_cells[index]
        for num in range(1, 10):
            if is_valid(board, row, col, num):
                board[row][col] = num
                if backtrack(index + 1):
                    return True
                board[row][col] = 0
        return False

    start = time.time()
    backtrack(0)
    end = time.time()
    return board, end - start

# Función para mostrar el Sudoku resuelto con los números originales en negrita y el marco
def print_sudoku_solution(original_board, solved_board):
    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    nums = [[""] + [f"\033[1m{solved_board[r][c]}\033[0m" if original_board[r][c] != 0 else str(solved_board[r][c]) for c in range(9)] for r in range(9)]

    print(line0)
    for r in range(1, side + 1):
        print("".join(n + s for n, s in zip(nums[r - 1], line1.split("."))))
        print([line2, line3, line4][(r % side == 0) + (r % base == 0)])

# Resolver Sudoku usando Backtracking
solved_board, time_taken = solve_backtracking(np.copy(puzzle_board))
print(f"\nResuelto en {time_taken:.4f} segundos")

# Mostrar el resultado final con los números originales en negrita y con el marco visual
print_sudoku_solution(puzzle_board, solved_board)



Resuelto en 0.0078 segundos
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 8 │ 3 │ 4 ║ 6 │ 1 │ 2 ║ 7 │ 5 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 7 │ 5 ║ 3 │ 4 │ 8 ║ 6 │ 1 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 1 ║ 7 │ 5 │ 9 ║ 3 │ 4 │ 8 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 5 │ 9 │ 6 ║ 8 │ 7 │ 4 ║ 2 │ 3 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 8 │ 7 ║ 2 │ 3 │ 1 ║ 9 │ 6 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 2 │ 3 ║ 9 │ 6 │ 5 ║ 8 │ 7 │ 4 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 4 │ 9 ║ 1 │ 8 │ 3 ║ 5 │ 2 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 1 │ 8 ║ 5 │ 2 │ 6 ║ 4 │ 9 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 5 │ 2 ║ 4 │ 9 │ 7 ║ 1 │ 8 │ 3 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


#Hill Climbing

In [ ]:
# Función para verificar si un número puede colocarse en una posición del tablero
def is_valid(board, row, col, num):
    if num in board[row]:
        return False
    if num in [board[i][col] for i in range(9)]:
        return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

# Función de Hill Climbing para resolver el Sudoku
def solve_hill_climbing(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def evaluate(board):
        """Evalúa cuántos conflictos hay en el tablero."""
        score = 0
        # Verificar filas
        for row in range(9):
            score += 9 - len(set(board[row]))
        # Verificar columnas
        for col in range(9):
            score += 9 - len(set([board[row][col] for row in range(9)]))
        return score

    def random_solution(board):
        """Llena el tablero con valores aleatorios donde hay celdas vacías."""
        for row, col in empty_cells:
            if board[row][col] == 0:
                board[row][col] = random.choice(range(1, 10))

    def hill_climbing():
        """Aplica hill climbing para intentar resolver el Sudoku."""
        current_board = np.copy(board)
        random_solution(current_board)  # Generar una solución inicial aleatoria
        current_score = evaluate(current_board)

        while current_score > 0:
            for row, col in empty_cells:
                best_score = current_score
                best_num = current_board[row][col]

                for num in range(1, 10):
                    if is_valid(current_board, row, col, num):
                        current_board[row][col] = num
                        new_score = evaluate(current_board)
                        if new_score < best_score:
                            best_score = new_score
                            best_num = num
                current_board[row][col] = best_num

            new_score = evaluate(current_board)
            if new_score == current_score:  # Si no hay mejora, devolver el resultado
                break
            current_score = new_score

        return current_board

    start = time.time()
    solved_board = hill_climbing()
    end = time.time()
    return solved_board, end - start

# Función para mostrar el Sudoku resuelto con los números originales en negrita y el marco
def print_sudoku_solution(original_board, solved_board):
    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    nums = [[""] + [f"\033[1m{solved_board[r][c]}\033[0m" if original_board[r][c] != 0 else str(solved_board[r][c]) for c in range(9)] for r in range(9)]

    print(line0)
    for r in range(1, side + 1):
        print("".join(n + s for n, s in zip(nums[r - 1], line1.split("."))))
        print([line2, line3, line4][(r % side == 0) + (r % base == 0)])

# Resolver Sudoku usando Hill Climbing
solved_board, time_taken = solve_hill_climbing(np.copy(puzzle_board))
print(f"\nResuelto en {time_taken:.4f} segundos")

# Mostrar el resultado final con los números originales en negrita y con el marco visual
print_sudoku_solution(puzzle_board, solved_board)



Resuelto en 0.0096 segundos
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 8 │ 3 │ 4 ║ 6 │ 1 │ 6 ║ 7 │ 5 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 7 │ 5 ║ 3 │ 2 │ 8 ║ 6 │ 1 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 1 ║ 7 │ 5 │ 9 ║ 3 │ 4 │ 8 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 5 │ 1 │ 6 ║ 9 │ 7 │ 4 ║ 2 │ 3 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 8 │ 7 ║ 2 │ 3 │ 1 ║ 9 │ 6 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 2 │ 3 ║ 5 │ 6 │ 7 ║ 8 │ 7 │ 1 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 4 │ 9 ║ 1 │ 8 │ 3 ║ 5 │ 2 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 1 │ 8 ║ 6 │ 8 │ 6 ║ 4 │ 9 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 5 │ 2 ║ 4 │ 9 │ 7 ║ 1 │ 8 │ 3 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


#A*

In [ ]:
import heapq

# Tamaños y símbolos del Sudoku
base  = 3  # Base de la cuadrícula 3x3
side  = base * base
symbol = " 1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Expande las líneas de la cuadrícula visual
def expandLine(line):
    return line[0] + line[5:9].join([line[1:5]*(base-1)]*base) + line[9:13]

# Función para verificar si un número puede colocarse en una posición del tablero
def is_valid(board, row, col, num):
    if num in board[row]:
        return False
    if num in [board[i][col] for i in range(9)]:
        return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

# Función heurística para calcular cuántas celdas vacías quedan
def heuristic(board):
    return sum(1 for row in range(9) for col in range(9) if board[row][col] == 0)

# Función A* para resolver el Sudoku
def solve_a_star(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def a_star():
        heap = []
        # La cola de prioridad usará (heurística, tablero, índice de celdas vacías)
        heapq.heappush(heap, (heuristic(board), board.tolist(), 0))  # Convertimos a lista

        while heap:
            h, current_board, index = heapq.heappop(heap)
            current_board = np.array(current_board)  # Convertimos de nuevo a array
            if index == len(empty_cells):
                return current_board  # Si hemos llenado todas las celdas vacías

            row, col = empty_cells[index]
            for num in range(1, 10):
                if is_valid(current_board, row, col, num):
                    new_board = current_board.copy()
                    new_board[row][col] = num
                    heapq.heappush(heap, (heuristic(new_board), new_board.tolist(), index + 1))

    start = time.time()
    solved_board = a_star()
    end = time.time()
    return solved_board, end - start

# Función para mostrar el Sudoku resuelto con los números originales en negrita y el marco
def print_sudoku_solution(original_board, solved_board):
    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    nums = [[""] + [f"\033[1m{solved_board[r][c]}\033[0m" if original_board[r][c] != 0 else str(solved_board[r][c]) for c in range(9)] for r in range(9)]

    print(line0)
    for r in range(1, side + 1):
        print("".join(n + s for n, s in zip(nums[r - 1], line1.split("."))))
        print([line2, line3, line4][(r % side == 0) + (r % base == 0)])

# Resolver Sudoku usando A*
solved_board, time_taken = solve_a_star(np.copy(puzzle_board))
print(f"\nResuelto en {time_taken:.4f} segundos")

# Mostrar el resultado final con los números originales en negrita y con el marco visual
print_sudoku_solution(puzzle_board, solved_board)



Resuelto en 0.0369 segundos
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 3 │ 5 ║ 2 │ 1 │ 9 ║ 7 │ 8 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 7 │ 6 ║ 3 │ 5 │ 4 ║ 2 │ 9 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 2 │ 1 ║ 7 │ 6 │ 8 ║ 3 │ 4 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 5 │ 8 │ 2 ║ 4 │ 7 │ 6 ║ 9 │ 1 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 9 │ 3 ║ 8 │ 2 │ 1 ║ 4 │ 5 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 4 │ 7 ║ 9 │ 3 │ 5 ║ 8 │ 6 │ 2 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 1 │ 4 ║ 6 │ 9 │ 2 ║ 5 │ 7 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 9 ║ 5 │ 8 │ 7 ║ 1 │ 3 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 5 │ 8 ║ 1 │ 4 │ 3 ║ 6 │ 2 │ 9 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


#Búsqueda Aleatoria

In [ ]:
import random

# Función para verificar si un número puede colocarse en una posición del tablero
def is_valid(board, row, col, num):
    if num in board[row]:
        return False
    if num in [board[i][col] for i in range(9)]:
        return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

# Función para llenar el tablero de manera aleatoria
def fill_randomly(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]
    for row, col in empty_cells:
        valid_nums = [num for num in range(1, 10) if is_valid(board, row, col, num)]
        if valid_nums:
            board[row][col] = random.choice(valid_nums)
        else:
            return False  # Si no hay números válidos, falla
    return True

# Función de búsqueda aleatoria para resolver el Sudoku
def solve_random_search(board, max_attempts=10000):
    attempts = 0
    start = time.time()
    while attempts < max_attempts:
        random_board = np.copy(board)
        if fill_randomly(random_board):
            end = time.time()
            return random_board, end - start
        attempts += 1
    return None, time.time() - start

# Función para mostrar el Sudoku resuelto con los números originales en negrita y el marco
def print_sudoku_solution(original_board, solved_board):
    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    nums = [[""] + [f"\033[1m{solved_board[r][c]}\033[0m" if original_board[r][c] != 0 else str(solved_board[r][c]) for c in range(9)] for r in range(9)]

    print(line0)
    for r in range(1, side + 1):
        print("".join(n + s for n, s in zip(nums[r - 1], line1.split("."))))
        print([line2, line3, line4][(r % side == 0) + (r % base == 0)])

# Resolver Sudoku usando Búsqueda Aleatoria
solved_board, time_taken = solve_random_search(np.copy(puzzle_board))
if solved_board is not None:
    print(f"\nResuelto en {time_taken:.4f} segundos")
    print_sudoku_solution(puzzle_board, solved_board)
else:
    print(f"\nNo se pudo resolver el Sudoku después de {time_taken:.4f} segundos y {10000} intentos.")


Resuelto en 0.4991 segundos
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 3 │ 5 ║ 2 │ 1 │ 9 ║ 7 │ 8 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 7 │ 6 ║ 3 │ 5 │ 4 ║ 2 │ 9 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 2 │ 1 ║ 7 │ 6 │ 8 ║ 3 │ 4 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 5 │ 8 │ 2 ║ 4 │ 7 │ 6 ║ 9 │ 1 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 9 │ 3 ║ 8 │ 2 │ 1 ║ 4 │ 5 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 4 │ 7 ║ 9 │ 3 │ 5 ║ 8 │ 6 │ 2 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 1 │ 4 ║ 6 │ 9 │ 2 ║ 5 │ 7 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 9 ║ 5 │ 8 │ 7 ║ 1 │ 3 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 5 │ 8 ║ 1 │ 4 │ 3 ║ 6 │ 2 │ 9 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


In [ ]:
import time
import numpy as np

# Función para ejecutar una heurística N veces y promediar su tiempo
def average_execution_time(heuristic_function, puzzle_board, N):
    total_time = 0
    for _ in range(N):
        # Crear una copia del puzzle para no modificar el original
        board_copy = np.copy(puzzle_board)
        # Ejecutar la heurística y medir el tiempo
        _, time_taken = heuristic_function(board_copy)
        total_time += time_taken
    # Calcular el tiempo promedio
    avg_time = total_time / N
    return avg_time

# Heurística de Backtracking
def solve_backtracking(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def backtrack(index):
        if index == len(empty_cells):
            return True
        row, col = empty_cells[index]
        for num in range(1, 10):
            if is_valid(board, row, col, num):
                board[row][col] = num
                if backtrack(index + 1):
                    return True
                board[row][col] = 0
        return False

    start = time.time()
    backtrack(0)
    end = time.time()
    return board, end - start

# Heurística de Hill Climbing
def solve_hill_climbing(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def evaluate(board):
        score = 0
        for row in range(9):
            score += 9 - len(set(board[row]))
        for col in range(9):
            score += 9 - len(set([board[row][col] for row in range(9)]))
        return score

    def random_solution(board):
        for row, col in empty_cells:
            if board[row][col] == 0:
                board[row][col] = random.choice(range(1, 10))

    def hill_climbing():
        current_board = np.copy(board)
        random_solution(current_board)
        current_score = evaluate(current_board)

        while current_score > 0:
            for row, col in empty_cells:
                best_score = current_score
                best_num = current_board[row][col]

                for num in range(1, 10):
                    if is_valid(current_board, row, col, num):
                        current_board[row][col] = num
                        new_score = evaluate(current_board)
                        if new_score < best_score:
                            best_score = new_score
                            best_num = num
                current_board[row][col] = best_num

            new_score = evaluate(current_board)
            if new_score == current_score:
                break
            current_score = new_score

        return current_board

    start = time.time()
    solved_board = hill_climbing()
    end = time.time()
    return solved_board, end - start

# Heurística de A*
def solve_a_star(board):
    empty_cells = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]

    def a_star():
        heap = []
        heapq.heappush(heap, (heuristic(board), board.tolist(), 0))

        while heap:
            h, current_board, index = heapq.heappop(heap)
            current_board = np.array(current_board)
            if index == len(empty_cells):
                return current_board

            row, col = empty_cells[index]
            for num in range(1, 10):
                if is_valid(current_board, row, col, num):
                    new_board = current_board.copy()
                    new_board[row][col] = num
                    heapq.heappush(heap, (heuristic(new_board), new_board.tolist(), index + 1))

    start = time.time()
    solved_board = a_star()
    end = time.time()
    return solved_board, end - start

# Heurística de Búsqueda Aleatoria
def solve_random_search(board, max_attempts=10000):
    attempts = 0
    start = time.time()
    while attempts < max_attempts:
        random_board = np.copy(board)
        if fill_randomly(random_board):
            end = time.time()
            return random_board, end - start
        attempts += 1
    return None, time.time() - start

# Función para validar las celdas del Sudoku (requerida por todas las heurísticas)
def is_valid(board, row, col, num):
    if num in board[row]:
        return False
    if num in [board[i][col] for i in range(9)]:
        return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

# Uso de la función average_execution_time
N = 20  # Número de veces que queremos repetir la heurística
print("Promedio de tiempos:")

# Ejecutar cada heurística N veces y promediar los tiempos
backtracking_avg_time = average_execution_time(solve_backtracking, puzzle_board, N)
hill_climbing_avg_time = average_execution_time(solve_hill_climbing, puzzle_board, N)
a_star_avg_time = average_execution_time(solve_a_star, puzzle_board, N)
random_search_avg_time = average_execution_time(solve_random_search, puzzle_board, N)

print(f"Backtracking: {backtracking_avg_time:.4f} segundos en promedio")
print(f"Hill Climbing: {hill_climbing_avg_time:.4f} segundos en promedio")
print(f"A*: {a_star_avg_time:.4f} segundos en promedio")
print(f"Búsqueda Aleatoria: {random_search_avg_time:.4f} segundos en promedio")


Promedio de tiempos:
Backtracking: 0.0373 segundos en promedio
Hill Climbing: 0.0165 segundos en promedio
A*: 0.0347 segundos en promedio
Búsqueda Aleatoria: 1.3531 segundos en promedio
